In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [5]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=postulaciones.columns[0])

In [6]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 3 columns):
idaviso             uint32
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint32(1)
memory usage: 126.0+ MB


# Obtengo postulantes no duplicados haber si asi aumento la cantidad

In [7]:
postulantes_en_postulaciones = postulaciones.drop_duplicates(subset='idpostulante', keep='first')

In [8]:
postulantes_en_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348883 entries, 0 to 6603750
Data columns (total 3 columns):
idaviso             348883 non-null uint32
idpostulante        348883 non-null object
fechapostulacion    348883 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint32(1)
memory usage: 9.3+ MB


In [9]:
postulantes_en_postulaciones.drop(['idaviso','fechapostulacion'],inplace=True,axis=1)

/home/lucio/Documentos/Datos/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['Unnamed: 0','nombre_numerico'])


In [11]:
postulantes_educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447909 entries, 0 to 447908
Data columns (total 3 columns):
idpostulante    447909 non-null object
nombre          447909 non-null category
estado          447909 non-null category
dtypes: category(2), object(1)
memory usage: 4.3+ MB


In [12]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'])


In [13]:
postulantes_genero_edad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante       504407 non-null object
fechanacimiento    504407 non-null object
sexo               504407 non-null category
dtypes: category(1), object(2)
memory usage: 8.2+ MB


In [14]:
postulantes_genero_edad.head()
#Hay que ver que hacer con los NO_DECLARA y sin fecha nacimiento

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,nan,NO_DECLARA
2,ZX1,nan,NO_DECLARA
3,Nq5,nan,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [16]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante       504407 non-null object
fechanacimiento    504407 non-null object
sexo               504407 non-null category
nombre             447909 non-null category
estado             447909 non-null category
dtypes: category(3), object(2)
memory usage: 13.0+ MB


In [17]:
#Agrego postulantes que no esten en el merge anterior
postulantes_merge2 = pd.merge(postulantes_merge,postulantes_en_postulaciones,how='outer')

In [18]:
postulantes_merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante       504407 non-null object
fechanacimiento    504407 non-null object
sexo               504407 non-null category
nombre             447909 non-null category
estado             447909 non-null category
dtypes: category(3), object(2)
memory usage: 13.0+ MB


# Avisos detalle

In [67]:
dtypes = pd.Series({'nombre_zona': CategoricalDtype(categories=['Buenos Aires (fuera de GBA)', 'Capital Federal',
                  'GBA Oeste', 'Gran Buenos Aires'],ordered=False),'ciudad': CategoricalDtype(categories=['Argentina', 'Barracas', 'Buenos Aires',
                  'Buenos Aires Province', 'CABA', 'Capital Federal', 'La Plata', 'Mendoza', 'Microcentro', 
                'Parque Patricios','República Argentina', 'San Isidro', 'Santa Rosa ','Tortuguitas',
                'Vicente Lopez', 'Zárate, Campana, Escobar','caba', 'paternal'], ordered=False),
                'nivel_laboral': CategoricalDtype(categories=['Gerencia / Alta Gerencia / Dirección',
                  'Jefe / Supervisor / Responsable', 'Junior', 'Otro', 'Senior / Semi-Senior'],
                 ordered=False),'tipo_de_trabajo': CategoricalDtype(categories=['Fines de Semana', 'Full-time', 'Part-time', 'Pasantia',
                  'Por Contrato', 'Por Horas', 'Primer empleo', 'Teletrabajo','Temporario'],ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

avisos_detalle = pd.read_csv('Data/Avisos-Detalle/Avisos-Detalle-merge.csv',dtype=column_types)
avisos_detalle = avisos_detalle.drop(columns=['Unnamed: 0'])

In [68]:
avisos_detalle.rename(index=str, columns={"idaviso": "idAviso"},inplace=True)
avisos_detalle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25288 entries, 0 to 25287
Data columns (total 11 columns):
idAviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null category
ciudad                  160 non-null category
mapacalle               2069 non-null object
tipo_de_trabajo         25288 non-null category
nivel_laboral           24953 non-null category
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: category(4), int64(2), object(5)
memory usage: 1.6+ MB


# Avisos online

In [69]:
avisos_online = pd.read_csv('Data/Avisos-Online/Avisos-Online-merge.csv')

In [70]:
avisos_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9886 entries, 0 to 9885
Data columns (total 1 columns):
idaviso    9886 non-null int64
dtypes: int64(1)
memory usage: 77.3 KB


# Vistas

In [71]:
dtypes = pd.Series({'idAviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

vistas = pd.read_csv('Data/Vistas/Vistas-merge.csv',dtype=column_types,parse_dates=['timestamp'],
                     infer_datetime_format=True)
vistas = vistas.drop(columns=['Unnamed: 0'])

In [72]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idAviso         uint32
timestamp       datetime64[ns]
idpostulante    object
dtypes: datetime64[ns](1), object(1), uint32(1)
memory usage: 164.0+ MB


# Quito duplicados para luego ver de que tipo es cada aviso
unique_advises = vistas.drop_duplicates(subset='idAviso', keep='first')

unique_advises.info()

# Agrego la informacion de avisos detalles
unique_advises = pd.merge(unique_advises,avisos_detalle,how='left',on='idAviso')

unique_advises = unique_advises[unique_advises['tipo_de_trabajo'].notnull()]
unique_advises.info()

unique_postulantes_vistas = vistas.drop_duplicates(subset='idpostulante', keep='first')

unique_postulantes_vistas.info()

groupby_aviso_postulante = vistas.groupby(['idpostulante','idAviso']).size()

asd = groupby_aviso_postulante['0002q']
aa = asd.index
aa.get_values()

In [73]:
# Creo funcion para recuperar lista de TIPOS DE TRABAJO de avisos que le interesan a un postulante de acuerdo
# a lo que visita y se postula

def obtener_lista_de_intereses(idpostulante):
    lista = []
    lista_de_ids_avisos_postulados = obtener_lista_de_ids_avisos_postulados(idpostulante)
    for id in lista_de_ids_avisos_postulados:
        tipo_de_trabajo = avisos_detalle[avisos_detalle['idAviso'] == id]['tipo_de_trabajo']
        lista.append(tipo_de_trabajo)
    lista_de_ids_avisos_visitados = obtener_lista_de_ids_avisos_visitados(idpostulante)
    if (lista_de_ids_avisos_visitados is not None):
        for id in lista_de_ids_avisos_visitados:
            tipo_de_trabajo = avisos_detalle[avisos_detalle['idAviso'] == id]['tipo_de_trabajo']
            lista.append(tipo_de_trabajo)    
    return lista
    
# Creo funcion para recuperar los ids de avisos a los que se postula un postulante
def obtener_lista_de_ids_avisos_postulados(idpostulante):
    data = postulaciones[postulaciones['idpostulante'] == idpostulante]
    return data['idaviso'].tolist()

# Creo funcion para recuperar los ids de avisos que visita un postulante
def obtener_lista_de_ids_avisos_visitados(idpostulante):
    try:
        return groupby_aviso_postulante.pop(idpostulante).index.get_values()
    except KeyError:
        return None

In [74]:
# Agrego columna con cantidad de postulaciones por postulante
serie_postulaciones = postulaciones['idpostulante'].value_counts()
def cant_postulaciones_postulante(idpostulante):
    try:
        return serie_postulaciones[idpostulante]
    except KeyError:
        return 0

In [75]:
groupby_aviso_postulante = vistas.groupby(['idpostulante','idAviso']).size()

In [76]:
postulantes_merge2['intereses'] = None
postulantes_merge2.loc[:,'intereses'] = postulantes_merge2['idpostulante'].apply(obtener_lista_de_intereses)

MemoryError: 

In [ ]:
postulantes_merge2['cant_postulaciones_a_avisos'] = 0
postulantes_merge2.loc[:,'cant_postulaciones_a_avisos'] = postulantes_merge2['idpostulante'].apply(cant_postulaciones_postulante)

# Avisos online

# Training test

In [ ]:
trainingSet = 

# Testing set

In [22]:
testingSet = pd.read_csv('Data/test_final_100k.csv')

In [49]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
id              100000 non-null int64
idaviso         100000 non-null int64
idpostulante    100000 non-null object
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


In [50]:
# Agrego informacion al set de testing con los datos que contamos

In [24]:
testingSet.drop_duplicates(subset='idaviso', keep='first')

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
3,3,758580,1Q35ej
6,6,776420,aZJ2XN
9,9,820850,6ZBD33
13,13,914880,6q6eNl
16,16,1112360,6kRZ44
20,20,1141070,edaNB4
23,23,1144960,NEJjvL
26,26,1145170,Y9d5Bw
29,29,1176260,ZvQJ2z


In [ ]:
dataframe = data_training3.drop(['price_aprox_usd'],axis=1)

columnas = columnas_corr
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento = data_training3.loc[(data_training3.year >= 2016)]
set_entrenamiento = set_entrenamiento[columnas_precio]
set_pruebas = data_training3.loc[(data_training3.year >= 2016)]

array = set_entrenamiento.values
X = array[:,0:7]
Y = array[:,7]

# prepare models
models = []
models.append(('LinearRegression', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('RidgeCV', RidgeCV()))
models.append(('Lasso', Lasso()))
models.append(('Elastic Net', ElasticNet()))
models.append(('LassoLars', LassoLars()))
models.append(('OrthogonalMatchingPursuit', OrthogonalMatchingPursuit()))
models.append(('BayesianRidge', BayesianRidge()))
models.append(('HuberRegressor  ', HuberRegressor()))

# evaluate each model in turn
for name, model in models:
    model.fit(X,Y)
    score = model.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    print("Precision = {:.2f} % , name = {}".format(score,name))